# Import packages

In [ ]:
import itertools

import matplotlib.pyplot as plt
import numpy as np

from stats import *
from utils import *
import visual as vi
from Inlier_Thresholder import Inlier_Thresholder
import scipy.io as spi
from time import time
import os
import seaborn as sns
from itertools import chain



# Initialize the paths to the files and variables containing the data

- **names**    contains the names of the files

- **mat_data**    dictionary with keys= file names ; values = files

In [ ]:
directory = 'C:/Users/carlo/IACV PROJECT/adelH'

# List all files in the directory
files = os.listdir(directory)

# Filter out only the .mat files
names = [file for file in files if file.endswith('.mat')]

# Load each .mat file
mat_data = {}
for file in names:
    file_path = os.path.join(directory, file)
    mat_data[file] = spi.loadmat(file_path)
    
names

# LMEDS inlier threshold computation and statistics

In [ ]:
# input parameter, how many files to analyse
number_of_data_to_analyse=len(names)



silhouette_scores = [[] for i in range(number_of_data_to_analyse)]
silhouette_avgs = [[] for i in range(number_of_data_to_analyse)]
labels_array = [[] for i in range(number_of_data_to_analyse)]
residuals = []
values_array = [[] for i in range(number_of_data_to_analyse)]
thresholds = [[] for i in range(number_of_data_to_analyse)]




for i in range(number_of_data_to_analyse):
    data=mat_data[names[i]]
    res=compute_inliers_residual_curve(data)
    residuals.append(res)

    for j in range(len(res)):
        anomaly_detector=Inlier_Thresholder(res[j], type="H")

        labels, threshold=anomaly_detector.use_best_method()  # use best method among the statistical ones, best according to silhouette
        
        if len(np.unique(labels))!=1:
            thresholds[i].append(threshold)
            labels_array[i].append(labels)
            silhouette_scr, silhouette_avg, values = silhouette_score_and_average(res[j], labels)
            silhouette_scores[i].append(silhouette_scr)
            silhouette_avgs[i].append(silhouette_avg)
            values_array[i].append(values)

In [ ]:
for i in range(len(silhouette_scores)):
    for j in range(len(silhouette_scores[i])):
        plot_silhouette(silhouette_scores[i][j], labels_array[i][j], silhouette_avgs[i][j], values_array[i][j], -thresholds[i][j])

# Plot statistics on inlier thresholds

In [ ]:
def plot_distribution_errors(errors):
    plt.figure(figsize=(8, 6))
    sns.histplot(errors, bins='auto',
                 kde=True)  # kde=True plots a kernel density estimate along with the histogram

    # Label the axes and add a title
    plt.xlabel("Projection Error Average")
    plt.ylabel("Density")
    plt.title("Distribution of Projection errors Scores LMEDS")

    plt.grid(True)
    plt.xlim(0, max(errors))

    # Display the plot
    plt.show()
    return 0

In [ ]:
# Create a histogram to visualize the distribution
plt.figure(figsize=(8, 6))
sns.histplot(list(chain.from_iterable(silhouette_avgs)), bins='auto', kde=True)  # kde=True plots a kernel density estimate along with the histogram

# Label the axes and add a title
plt.xlabel("Silhouette Score")
plt.ylabel("Density")
plt.title("Distribution of Silhouette Scores")

plt.grid(True)
plt.xlim(-1, 1)  # Set limits for silhouette scores (typically between -1 and 1)

# Display the plot
plt.show()

In [ ]:
# Create a histogram to visualize the distribution
plt.figure(figsize=(8, 6))
sns.histplot(list(chain.from_iterable(thresholds)), bins='auto', kde=True)  # kde=True plots a kernel density estimate along with the histogram

# Label the axes and add a title
plt.xlabel("Inlier thresholds")
plt.ylabel("Density")
plt.title("Distribution of Inlier Thresholds")

plt.grid(True)
plt.xlim(0, max(max(thresholds)))  # Set limits for silhouette scores (typically between -1 and 1)

# Display the plot
plt.show()

In [ ]:
#mean and variance of the inlier thresholds distribution
mean = np.mean(np.abs(list(chain.from_iterable(thresholds))))
var = np.var(np.abs(list(chain.from_iterable(thresholds))))
median = st.median(np.abs(list(chain.from_iterable(thresholds))))




# median thresholds
med_thresholds=[st.median(t) for t in thresholds]


print(f"The threshold distribution has mean: {mean} and variance: {var}")
print(f"The threshold distribution has median: {median}")

# Outlier detection using new inlier thresholds with GC RANSAC

### Plot residual matrices and outliers using gc-ransac


In [ ]:
number_of_data_to_analyse=len(names)
plot_heat=True

for i in range(number_of_data_to_analyse):
    data=mat_data[names[i]]
    res=build_residual_matrix(data, plot=True, method="gc-ransac" , threshold=thresholds[i], verbose=True)
    
    labl=mat_data[names[i]]["label"]
    mod_lab=np.where(labl>0)
    labl=labl[mod_lab].reshape(-1,1)
    labl=labl/np.max(labl)
    labl=labl*np.max(res)
    
    if plot_heat:
        plot_residual_matrix(res,labl)

    

# Analysis of residual curves of GC RANSAC with new thresholds

In [ ]:
# input parameter, how many files to analyse
number_of_data_to_analyse=len(names)



silhouette_scores_gc = [[] for i in range(number_of_data_to_analyse)]
silhouette_avgs_gc = [[] for i in range(number_of_data_to_analyse)]
labels_array_gc = [[] for i in range(number_of_data_to_analyse)]
residuals_gc = []
values_array_gc = [[] for i in range(number_of_data_to_analyse)]




for i in range(number_of_data_to_analyse):
    data=mat_data[names[i]]
    
    img1, img2 = data["img1"], data["img2"]

    outliers, models = vi.group_models(data)["outliers"], vi.group_models(data)["models"]
    
    points=extract_points(models,data)
    
    labels = points[3]
    
    points=points[0]
    
    res=build_residual_matrix(data, plot=False, verbose=False,method="gc-ransac" , threshold=thresholds[i])


    for j in range(len(models)):
        
        src = points["src_points"][j]
        dst = points["dst_points"][j]
        
        _,lbl=verify_pygcransac_H(src,dst,img1,img1,med_thresholds[i], verbose=False)
         
        lbl=1-lbl
        
        residuals_of_model=res[np.where(labels==j+1),j]
        
        sort_index=np.argsort(residuals_of_model.ravel())
        
        sorted_res=residuals_of_model.ravel()[sort_index]
        
        sorted_lbl=lbl[sort_index]
        
        
        
        if len(np.unique(sorted_lbl))!=1:
            labels_array_gc[i].append(sorted_lbl)
            silhouette_scr, silhouette_avg, values = silhouette_score_and_average(sorted_res, sorted_lbl)
            silhouette_scores_gc[i].append(silhouette_scr)
            silhouette_avgs_gc[i].append(silhouette_avg)
            values_array_gc[i].append(values)

### Residual curves of gc ransac

In [ ]:
for i in range(len(silhouette_scores_gc)):
    for j in range(len(silhouette_scores_gc[i])):
        plot_silhouette(silhouette_scores_gc[i][j], labels_array_gc[i][j], silhouette_avgs_gc[i][j], values_array_gc[i][j], -med_thresholds[i])

### Residual curves of LMEDS

In [ ]:
for i in range(len(silhouette_scores)):
    for j in range(len(silhouette_scores[i])):
        plot_silhouette(silhouette_scores[i][j], labels_array[i][j], silhouette_avgs[i][j], values_array[i][j], -thresholds[i][j])

### AVG silhouette score LMEDS vs GC RANSAC

In [ ]:
flatten_avgs_LMEDS=[value for  sublist in silhouette_avgs for value in sublist]
flatten_avgs_GCRANSAC=[value for  sublist in silhouette_avgs_gc for value in sublist]

print("LMEDS avarage silhhouette: " , np.mean(flatten_avgs_LMEDS))
print("GC-RANSAC avarage silhhouette: " , np.mean(flatten_avgs_GCRANSAC))

